In [4]:
from typing import List, Dict, Optional

def format_reflections(reflections: List[Dict[str, str]]) -> str:
    """
    Formats a list of reflections into a readable string
    
    Args:
        reflections: List of dictionaries with 'question' and 'answer' keys
        
    Returns:
        Formatted string with each reflection numbered
        
    Raises:
        ValueError: If reflections list is empty
    """
    if not reflections:
        raise ValueError("Reflections list cannot be empty")
    
    formatted = []
    for i, reflection in enumerate(reflections, 1):
        question = reflection.get('question', 'No question provided')
        answer = reflection.get('answer', 'No answer provided')
        formatted.append(
            f"Reflection {i}:\n"
            f"Question: {question}\n"
            f"Answer: {answer}"
        )
    
    return "\n\n".join(formatted)



In [5]:
example_reflections = [
        {"question": "How do I handle stress?", "answer": "I tend to avoid stressful situations"},
        {"question": "My communication style?", "answer": "I listen more than I speak"}
    ]

In [6]:
my_promt = format_reflections(example_reflections)

In [7]:
my_promt

'Reflection 1:\nQuestion: How do I handle stress?\nAnswer: I tend to avoid stressful situations\n\nReflection 2:\nQuestion: My communication style?\nAnswer: I listen more than I speak'

In [11]:
import os
from dotenv import load_dotenv
from typing import List, Dict

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough

# Load environment variables
load_dotenv()

# Pydantic Models for Structured Output
class Insight(BaseModel):
    thinking: str = Field(..., description="Analysis of your thought patterns")
    blindspot: str = Field(..., description="What you might be overlooking")
    growth: str = Field(..., description="Your growth opportunity")
    action: str = Field(..., description="Your recommended next step")

class Insights(BaseModel):
    insights: List[Insight]

# Initialize the model
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", 
    temperature=0, 
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Create Output Parser
parser = PydanticOutputParser(pydantic_object=Insights)

# Define Prompt Template
insight_template = """
You're my personal reflection coach. I'm sharing these thoughts with you:

{reflections}

For each entry I've shared, please help me understand:
1. What my thinking patterns reveal about me
2. What perspectives I might be missing
3. Where I have the most potential to grow
4. One concrete action I could take

Format your response as clean JSON with these exact fields for each insight:
- "thinking"
- "blindspot" 
- "growth"
- "action"

Structure the output like this:
{{
  "insights": [
    {{
      "thinking": "...",
      "blindspot": "...", 
      "growth": "...",
      "action": "..."
    }}
  ]
}}
"""

# Create Prompt Template
insight_prompt = PromptTemplate(
    template=insight_template,
    input_variables=["reflections"]
)

# Create the final chain
reflection_insights_chain = (
    {"reflections": RunnablePassthrough()} 
    | insight_prompt 
    | model 
    | parser
)

In [12]:
reflection_insights_chain.invoke(my_promt)

Insights(insights=[Insight(thinking='Your response to stress reveals a pattern of avoidance. This suggests a preference for maintaining comfort and minimizing discomfort, potentially prioritizing short-term relief over long-term growth and resilience.', blindspot='By avoiding stressful situations, you might be missing opportunities for personal growth, skill development, and building resilience.  You may also be inadvertently allowing stressors to accumulate, leading to larger problems down the line.  You might not be developing effective coping mechanisms.', growth='Your potential for growth lies in developing healthier coping mechanisms for stress. This includes learning to identify and manage stressors proactively, rather than avoiding them.  Building resilience will allow you to navigate challenges more effectively.', action='Identify one upcoming stressful situation (e.g., a difficult conversation, a deadline) and create a plan to address it head-on.  This plan should include spec